# Validation et importation des données d'interaction utilisateur-article <a class="anchor" id="top"></a>

Dans ce bloc-notes, vous allez choisir un jeu de données et le préparer être utilisé sur Amazon Personalize.

1. [Introduction](#intro)
1. [Choisir un jeu de données ou une source de données](#source)
1. [Préparer vos données](#prepare)
1. [Créer des groupes de jeux de données et le jeu de données d'interactions](#group_dataset)
1. [Configurer un compartiment S3 et un rôle IAM](#bucket_role)
1. [Importer les données d'interactions](#import)

## Introduction <a class="anchor" id="intro"></a>

Pour la plupart, les algorithmes d'Amazon Personalize (appelés recettes) cherchent à résoudre différentes tâches, expliquées ici :

1. **Personnalisation de l'utilisateur** – Nouvelle version qui prend en charge TOUS les flux HRNN/besoins de personnalisation de l'utilisateur ; ce sera ce que nous utilisons ici.
1. **HRNN et métadonnées HRNN** – Recommande des articles en fonction des interactions précédentes de l'utilisateur avec les articles.
1. **HRNN-Coldstart** – Recommande de nouveaux articles pour lesquels les données d'interaction ne sont pas encore disponibles.
1. **Classement personnalisé** – Prend une collection d'articles et les ordonne dans l'ordre probable d'intérêt en utilisant une approche de type HRNN.
1. **SIMS (Éléments similaires)** – À partir d'un article donné, recommande d'autres articles avec lesquels les utilisateurs ont également interagi.
1. **Compte de popularité** – Recommande les articles les plus populaires, si HRNN ou métadonnées HRNN n'ont pas de réponse – cela est renvoyé par défaut.

Quel que soit le cas d'utilisation, les algorithmes partagent tous une base d'apprentissage sur les données d'interaction entre l'utilisateur et l'article, qui sont définies par trois attributs fondamentaux :

1. **UserID** – L'utilisateur qui a interagi
1. **ItemID** – Article avec lequel l'utilisateur a interagi
1. **Horodatage** – L'heure à laquelle l'interaction s'est produite

Nous prenons également en charge les types d'événements et les valeurs d'événements définis par :

1. **Type d'événement​​** – Étiquette catégorique d'un événement (parcourir, acheté, évalué, etc.).
1. **Valeur de l'événement** – Une valeur correspondant au type d'événement qui s'est produit. En règle générale, nous recherchons des valeurs normalisées entre 0 et 1 relatives aux types d'événements. Par exemple, s'il y a trois phases pour terminer une transaction (cliqué, ajouté au panier et acheté), alors il y aura une valeur event_value pour chaque phase comme 0,33, 0,66 et 1,0 respectivement.

Les champs type d'événement et valeur d'événement sont des données supplémentaires qui peuvent être utilisées pour filtrer les données envoyées en vue de l'entraînement du modèle de personnalisation. Dans cet exercice particulier, nous n'aurons pas de type d'événement ou de valeur d'événement. 

## Choisir un jeu de données ou une source de données <a class="anchor" id="source"></a>
[Retour au début](#top)

Comme nous l'avons mentionné, les données d'interaction utilisateur-article sont essentielles pour commencer à utiliser le service. Cela signifie que nous devons rechercher les cas d'utilisation qui génèrent ce type de données, dont voici quelques exemples courants :

1. Applications de vidéo à la demande
1. Plateformes d'e-commerce
1. Agrégateurs/plateformes de médias sociaux

Il existe quelques lignes directrices pour définir la portée d'un problème adapté à Personalize. Nous recommandons les valeurs ci-dessous comme point de départ, bien que les [limites officielles](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) soient un peu plus basses.

* Utilisateurs authentifiés
* Au moins 50 utilisateurs uniques
* Au moins 100 articles uniques
* Au moins deux douzaines d'interactions pour chaque utilisateur 

La plupart du temps, il est facile d'y parvenir, et si vous êtes dans une catégorie faible, vous pouvez souvent compenser en ayant un chiffre plus élevé dans une autre catégorie.

En règle générale, vos données n'arriveront pas sous une forme parfaite pour Personalize, et il faudra les modifier afin qu'elles soient structurées correctement. Ce bloc-notes a pour objectif de vous guider dans tout ceci. 

Pour commencer, nous allons utiliser le dernier jeu de données MovieLens. Ce jeu de données comprend plus de 25 millions d'interactions et une riche collection de métadonnées pour les articles. Il existe également une version plus petite de ce jeu de données, qui peut être utilisée pour raccourcir les temps d'entraînement, tout en intégrant toujours les mêmes capacités que le jeu de données complet. Réglez USE_FULL_MOVIELENS sur True pour utiliser le jeu de données complet.

In [ ]:
USE_FULL_MOVIELENS = False

Tout d'abord, vous allez télécharger le jeu de données et le décompresser dans un nouveau dossier en utilisant le code ci-dessous.

In [ ]:
data_dir = "poc_data"
!mkdir $data_dir

if not USE_FULL_MOVIELENS:
    !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
    !cd $data_dir && unzip ml-latest-small.zip
    dataset_dir = data_dir + "/ml-latest-small/"
else:
    !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-25m.zip
    !cd $data_dir && unzip ml-25m.zip
    dataset_dir = data_dir + "/ml-25m/"

Examinez les fichiers de données que vous avez téléchargés.

In [ ]:
!ls $dataset_dir

À l'heure actuelle, nous ne savons pas grand-chose, sauf que nous avons quelques CSV et un readme. Ensuite, nous allons extraire le fichier readme pour en savoir plus !

In [ ]:
!pygmentize $dataset_dir/README.txt

D'après le fichier README, nous voyons qu'un fichier `ratings.csv` devrait fonctionner comme un proxy pour nos données d'interactions ; après tout, l'évaluation d'un film est certainement une forme d'interaction avec celui-ci. Le jeu de données contient également des informations sur le genre comme des données sur le génome du film. Dans cette POC, nous nous concentrerons sur les interactions et les données de genre.


## Préparer vos données <a class="anchor" id="prepare"></a>
[Retour au début](#top)

Chargez ensuite les données et assurez-vous qu'elles sont en bon état, puis enregistrez-les dans un CSV où elles sont prêtes à être utilisées avec Amazon Personalize.

Pour commencer, importez une collection de bibliothèques Python couramment utilisées en science des données.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

Ensuite, ouvrez le fichier de données et examinez les premières lignes.

In [ ]:
original_data = pd.read_csv(dataset_dir + '/ratings.csv')
original_data.head(5)

In [ ]:
original_data.shape

In [ ]:
original_data.describe()

Cela montre que nous avons une bonne gamme de valeurs pour `userId` et `movieId`. Ensuite, il est toujours bon de vérifier le format des données.

In [ ]:
original_data.info()

In [ ]:
original_data.isnull().any()

Vous pouvez donc voir qu'il y a un total de (25 000 095 pour le jeu complet, 100 836 pour le jeu limité) entrées dans le jeu de données, avec 4 colonnes, et chaque cellule stockée au format int64, à l'exception de la cote qui est un float64.

Le format int64 est clairement adapté à `userId` et `movieId`. Cependant, nous effectuer une analyse plus approfondie pour comprendre les horodatages dans les données. Pour utiliser Amazon Personalize, vous devez enregistrer les horodatages au format [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time).

Actuellement, les valeurs d'horodatage ne sont pas lisibles par l'homme. Prenons donc une valeur d'horodatage arbitraire et voyons comment l'interpréter.

Effectuez un test d'intégrité du jeu de données transformé en choisissant un horodatage arbitraire et en le transformant en un format lisible par l'homme.

In [ ]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

Cette date a du sens en tant qu'horodatage. Nous pouvons donc continuer à formater le reste des données. Rappelez-vous que les données dont nous avons besoin sont des données d'interaction utilisateur-article, qui sont `userId`, `movieId` et `timestamp` dans ce cas. Notre jeu de données comporte une colonne supplémentaire, `rating`, qui peut être supprimée après que nous l'ayons exploitée pour nous concentrer sur les interactions positives.

Étant donné qu'il s'agit d'un jeu de données d'avis explicite sur des films, il comprend des films notés de 1 à 5, nous voulons inclure uniquement les mouvements qui ont été "aimés" par les utilisateurs, et simuler un jeu de données implicites qui ressemble plus à ce que les données seraient recueillies par une plate-forme de VOD. Pour ce faire, nous allons filtrer toutes les interactions inférieures à 2 sur 5, et créer un EVENT_Type "clic" et un EVENT_Type "regarder". Nous affecterons ensuite tous les films classés 2 et plus comme "clic" et les films au-dessus de 4 comme "clic" et "visionnage".

Notez que cela doit correspondre aux événements que nous modélisons. Pour un véritable jeu de données que vous modéliseriez en vous basant sur des retours implicites comme des clics, des visions et/ou des retours explicites comme des évaluations, des likes, etc.

In [ ]:
watched_df = original_data.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'movieId', 'timestamp']]
watched_df['EVENT_TYPE']='watch'
watched_df.head()

In [ ]:
clicked_df = original_data.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'movieId', 'timestamp']]
clicked_df['EVENT_TYPE']='click'
clicked_df.head()

In [ ]:
interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 

In [ ]:
interactions_df.info()

regardons à quoi ressemble le nouveau jeu de données 

In [ ]:
interactions_df.describe()

Après avoir manipulé les données, vérifiez toujours si le format des données a changé.

In [ ]:
interactions_df.dtypes

 Amazon Personalize utilise des noms de colonne par défaut pour les utilisateurs, les articles et l'horodatage. Ces noms de colonnes par défaut sont `USER_ID`, `ITEM_ID` ET `TIMESTAMP`. La dernière modification apportée au jeu de données consiste donc à remplacer les en-têtes de colonnes existants par les en-têtes par défaut.

In [ ]:
interactions_df.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

Voilà ! À ce stade, les données sont prêtes à être utilisées et il ne nous reste plus qu'à les enregistrer dans un fichier CSV.

In [ ]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## Créer des groupes de jeux de données et le jeu de données d'interactions <a class="anchor" id="group_dataset"></a>
[Retour au début](#top)

Le plus haut niveau d'isolation et d'abstraction avec Amazon Personalize est le *groupe de jeux de données*. Toute information stockée dans l'un de ces groupes de jeux de données n'a aucun impact sur un autre groupe de données ou sur les modèles créés à partir de celui-ci – ils sont entièrement indépendants. Vous pouvez ainsi réaliser de nombreuses expériences, et c'est en partie grâce à cela que vos modèles restent privés et ne sont entraînés que sur vos données. 

Avant d'importer les données préparées antérieurement, il faut créer un groupe de jeux de données et y ajouter un jeu de données qui gère les interactions.

Les groupes de jeux de données peuvent comporter les types d'informations suivants :

* Interactions utilisateur-article
* Flux d'événements (interactions en temps réel)
* Métadonnées de l'utilisateur
* Métadonnées de l'article

Avant de créer le groupe de jeux de données et le jeu de données pour nos données d'interaction, vérifions que votre environnement peut communiquer avec Amazon Personalize.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Créer le groupe de jeux de données

La cellule suivante va créer un nouveau groupe de jeux de données avec le nom `personalize-poc-lastfm`.

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-poc-movielens"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

Pour pouvoir utiliser le groupe de jeux de données, il doit être actif. Cela peut prendre une ou deux minutes. Exécutez la cellule ci-dessous et attendez qu'elle affiche le statut ACTIF. Il vérifie l'état du groupe de jeux de données toutes les secondes, jusqu'à un maximum de 3 heures.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Maintenant que vous avez un groupe de jeux de données, vous pouvez créer un jeu de données pour les données d'interaction.

### Créer le jeu de données

Tout d'abord, définissez un schéma afin d'indiquer à Amazon Personalize le type de jeu de données que vous chargez. Plusieurs mots-clés réservés et obligatoires sont requis dans le schéma, en fonction du type de jeu de données. Des informations plus détaillées peuvent être retrouvées dans la [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Ici, vous allez créer un schéma pour les données d'interactions, qui nécessite les champs `USER_ID`, `ITEM_ID` et `TIMESTAMP`  Ils doivent être définis dans le schéma en respectant l'ordre dans lequel ils apparaissent dans le jeu de données.

In [ ]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-poc-movielens-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

Une fois le schéma créé, vous pouvez créer un jeu de données dans le groupe de jeux de données. Il est à noter que cette opération ne charge pas encore les données. Cela arrivera quelques étapes plus tard.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-poc-movielens-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Configurer un compartiment S3 et un rôle IAM <a class="anchor" id="bucket_role"></a>
[Retour au début](#top)

Pour l'instant, nous avons téléchargé, manipulé et enregistré les données sur l'instance Amazon EBS associée à l'instance qui exécute ce bloc-notes Jupyter. Toutefois, Amazon Personalize aura besoin d'un compartiment S3 pour servir de source à vos données, ainsi que de rôles IAM pour accéder à ce compartiment. Mettons tout cela en place.

Utilisez les métadonnées stockées sur l'instance sous-jacente de ce bloc-notes Amazon SageMaker pour déterminer la région dans laquelle il fonctionne. Si vous utilisez un bloc-notes Jupyter en dehors d'Amazon SageMaker, définissez simplement la région sous forme de chaîne comme suit. Le compartiment Amazon S3 doit se trouver dans la même région que les ressources Amazon Personalize que nous avons créées jusqu'à présent.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

Les noms des compartiments Amazon S3 sont uniques au niveau mondial. Pour créer un nom de compartiment unique, le code ci-dessous ajoutera la chaîne de caractères `personalizepocvod` à votre numéro de compte AWS. Puis il crée un compartiment avec ce nom dans la région découverte dans la cellule précédente.

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizepocvod"
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
        )

### Charger les données vers S3

Maintenant que votre compartiment Amazon S3 a été créé, chargez le fichier CSV de nos données d'interaction utilisateur-article. 

In [ ]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### Définir la politique du compartiment S3
Amazon Personalize doit pouvoir lire le contenu de votre compartiment S3. Il faut donc ajouter une politique de compartiment qui l'autorise.

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

### Créer un rôle IAM

Amazon Personalize doit pouvoir assumer des rôles dans AWS, afin de disposer des autorisations nécessaires pour exécuter certaines tâches. Créons un rôle IAM et ajoutons-lui les politiques requises. Le code ci-dessous associe des politiques très permissives ; veuillez utiliser des politiques plus restrictives pour toute application de production.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRolePOC"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## Importer les données d'interactions <a class="anchor" id="import"></a>
[Retour au début](#top)

Précédemment, vous avez créé le groupe de jeux de données et le jeu de données pour héberger vos informations. Vous allez maintenant exécuter une tâche d'importation qui chargera les données du compartiment S3 dans le jeu de données Amazon Personalize. 

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-import1",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

Pour pouvoir utiliser le jeu de données, la tâche d'importation doit être active. Exécutez la cellule ci-dessous et attendez qu'elle affiche le statut ACTIF. Il vérifie l'état de la tâche d'importation toutes les secondes, jusqu'à un maximum de 6 heures.

L'importation des données peut prendre un certain temps, en fonction de la taille du jeu de données. Dans cet atelier, l'importation des données devrait durer environ 15 minutes.

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Lorsque l'importation du jeu de données est active, vous êtes prêt à commencer à créer des modèles avec SIMS, Classement personnalisé, Compte de popularité et Personnalisation de l'utilisateur. Ce processus se poursuivra dans d'autres blocs-notes. Exécutez la cellule ci-dessous avant de passer au stockage de quelques valeurs à utiliser dans les prochains blocs-notes.

In [ ]:
%store USE_FULL_MOVIELENS
%store dataset_dir
%store interactions_dataset_arn
%store dataset_group_arn
%store bucket_name
%store role_arn
%store role_name
%store data_dir
%store region
%store interaction_schema_arn